In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

start_date = datetime.strptime("1957-03-01", "%Y-%m-%d")
end_date = datetime.strptime("2016-12-31", "%Y-%m-%d")
plt.style.use('ggplot')

from google.colab import drive
drive.mount('drive')

Mounted at drive


# Data Cleaning

In [ ]:
# Define column names to exclude
exception_names = ["permno", "SHROUT", "mve0", "prc"]

chunksize = 10 ** 6
list_of_dfs = []
for chunk in pd.read_csv("drive/MyDrive/MATH5470/datashare/GKX_20201231.csv", chunksize=chunksize):
    # chunk is a DataFrame. To "process" the rows in the chunk:
    chunk['DATE'] = pd.to_datetime(chunk['DATE'], format="%Y%m%d")

    mask = (chunk['DATE'] >= start_date) & (chunk['DATE'] <= end_date)
    list_of_dfs.append(chunk.loc[mask])
    print(list_of_dfs[-1].shape)
original_data = pd.concat(list_of_dfs)
del list_of_dfs

(691358, 101)
(1000000, 101)
(1000000, 101)
(1000000, 101)
(68675, 101)


In [ ]:
original_data.head(10)

permno       DATE       mvel1       RET     prc  SHROUT      beta  \
308642   10006 1957-03-29   86308.500  0.018405  62.250    1412  1.117907   
308643   10014 1957-03-29    3903.375  0.000000   2.625    1487  0.331304   
308644   10022 1957-03-29    9841.000 -0.003846   6.375    1514  0.942052   
308645   10030 1957-03-29   51901.000  0.077907  45.750    1207  0.886532   
308646   10057 1957-03-29   35250.000 -0.017730  69.250     500  1.229520   
308647   10065 1957-03-29   76945.250 -0.010309  24.000    3173  1.218324   
308648   10102 1957-03-29  188958.750  0.048780  53.750    3687  1.476413   
308649   10137 1957-03-29  235620.000  0.033333  27.125    8976  0.621582   
308650   10145 1957-03-29  867562.500 -0.017143  86.000    9915  1.115002   
308651   10153 1957-03-29  266617.750  0.011450  33.125    8141  1.139526   

          betasq     chmom     dolvol  ...  baspread           ill    maxret  \
308642  1.249717  0.134574  10.296745  ...  0.013856  1.557681e-07  0.029167   
308643  0.109762  0.006667   7.032404  ...  0.031389  8.383815e-06  0.071429   
308644  0.887461  0.058761   7.294038  ...  0.010066  5.360386e-06  0.020000   
308645  0.785940 -0.032065   9.516942  ...  0.009523  1.090908e-07  0.017857   
308646  1.511719 -0.102811   8.421013  ...  0.007661  2.386914e-07  0.007194   
308647  1.484313  0.161162   9.451226  ...  0.011691  1.942569e-07  0.015789   
308648  2.179796 -0.029672  10.582834  ...  0.015330  7.570978e-08  0.027638   
308649  0.386364  0.016174   9.261984  ...  0.006887  9.263915e-08  0.009302   
308650  1.243229 -0.021979  10.977807  ...  0.014094  1.351773e-07  0.014493   
308651  1.298520  0.000253  10.736065  ...  0.014661  5.995544e-08  0.019455   

          retvol  std_dolvol  std_turn     zerotrade  sic2  bm  bm_ia  
308642  0.013546    0.914592  0.802461  1.066005e-07  37.0 NaN    NaN  
308643  0.031165    0.943409  0.716406  1.105263e+00   NaN NaN    NaN  
308644  0.016886    0.993558  0.534808  2.387797e-07   NaN NaN    NaN  
308645  0.009104    0.736000  0.712647  9.202902e-08   NaN NaN    NaN  
308646  0.006748    0.889503  0.775954  6.631579e+00   NaN NaN    NaN  
308647  0.009285    0.585667  0.294445  2.372159e-07   NaN NaN    NaN  
308648  0.014074    0.616426  0.773493  1.109778e-07  28.0 NaN    NaN  
308649  0.005742    0.489383  0.124105  5.043385e-07  49.0 NaN    NaN  
308650  0.009338    0.990992  0.150461  4.488909e-07   NaN NaN    NaN  
308651  0.010043    0.407484  0.210700  2.159646e-07  13.0 NaN    NaN  

[10 rows x 101 columns]

In [ ]:
original_data.shape

(3760033, 101)

In [ ]:
original_macro_preditors = pd.read_excel("drive/MyDrive/MATH5470/datashare/PredictorData2023.xlsx", na_values="NaN")
original_macro_preditors['yyyymm'] = pd.to_datetime(original_macro_preditors['yyyymm'], format="%Y%m")
mask = (original_macro_preditors['yyyymm'] >= start_date) & (original_macro_preditors['yyyymm'] <= end_date)
original_macro_preditors = original_macro_preditors.loc[mask]
original_macro_preditors.head(10)

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


yyyymm  Index   D12      E12       b/m     tbl     AAA     BAA  \
1034 1957-03-01  44.11  1.73  3.40000  0.599819  0.0308  0.0366  0.0443   
1035 1957-04-01  45.74  1.73  3.40667  0.576098  0.0307  0.0367  0.0444   
1036 1957-05-01  47.43  1.73  3.41333  0.564039  0.0306  0.0374  0.0452   
1037 1957-06-01  47.37  1.73  3.42000  0.565877  0.0329  0.0391  0.0463   
1038 1957-07-01  47.91  1.74  3.43667  0.560057  0.0316  0.0399  0.0473   
1039 1957-08-01  45.22  1.75  3.45333  0.588005  0.0337  0.0410  0.0482   
1040 1957-09-01  42.42  1.76  3.47000  0.624151  0.0353  0.0412  0.0493   
1041 1957-10-01  41.06  1.77  3.43667  0.645746  0.0358  0.0410  0.0499   
1042 1957-11-01  41.72  1.78  3.40333  0.633072  0.0331  0.0408  0.0509   
1043 1957-12-01  39.99  1.79  3.37000  0.653676  0.0304  0.0381  0.0503   

         lty      ntis   Rfree      infl     ltr   corpr      svar       csp  \
1034  0.0331  0.026600  0.0023  0.003610 -0.0024  0.0050  0.000330  0.001054   
1035  0.0345  0.027421  0.0025  0.003597 -0.0222 -0.0066  0.000302  0.002142   
1036  0.0348  0.028849  0.0026  0.003584 -0.0023 -0.0075  0.000482  0.002422   
1037  0.0361  0.030528  0.0024  0.003571 -0.0180 -0.0322  0.000579  0.002176   
1038  0.0365  0.032346  0.0030  0.007117 -0.0041 -0.0110  0.000554  0.001631   
1039  0.0367  0.035854  0.0025  0.000000  0.0002 -0.0009  0.002147  0.001350   
1040  0.0364  0.034363  0.0026  0.000000  0.0076  0.0095  0.001517  0.001117   
1041  0.0369  0.038410  0.0029  0.000000 -0.0050  0.0023  0.004936  0.001123   
1042  0.0340  0.040990  0.0028  0.003534  0.0533  0.0311  0.003490  0.000804   
1043  0.0323  0.034873  0.0024  0.000000  0.0307  0.0685  0.001251  0.000668   

      CRSP_SPvw  CRSP_SPvwx  
1034   0.023827    0.020752  
1035   0.046538    0.045215  
1036   0.038734    0.033208  
1037  -0.000705   -0.003632  
1038   0.009747    0.008217  
1039  -0.048998   -0.054178  
1040  -0.057879   -0.060609  
1041  -0.038834   -0.040549  
1042   0.023703    0.016483  
1043  -0.037422   -0.041021

In [ ]:
# Define function to replace missing values with column median
def replace_nas(x):
    med = x.median(skipna=True)
    x = x.fillna(med)
    x = x.fillna(0)
    return x

# Define function to perform rank transformation
def rank_transform(x):
    rx = x.rank(na_option="keep",pct=True)
    return 2*(rx - 0.5)


In [ ]:
# Convert DATE column to datetime and floor to nearest month
original_data["DATE"] = original_data["DATE"].dt.to_period("M").dt.to_timestamp()

# Define column names to exclude
exception_names = ["permno", "DATE", "RET", "SHROUT", "sic2", "mve0", "prc"]

# Apply missing value replacement and rank transformation to relevant columns in characteristics data
characteristics_names = [col for col in original_data.columns if col not in exception_names]
original_data[characteristics_names] = original_data.groupby(["DATE"], as_index=False, group_keys=False)[characteristics_names].apply(rank_transform)
original_data[characteristics_names] = original_data.groupby(["DATE"], as_index=False, group_keys=False)[characteristics_names].apply(replace_nas)



In [ ]:
# Convert yyyymm column to datetime and floor to nearest month, then rename
original_macro_preditors["DATE"] = original_macro_preditors["yyyymm"].dt.to_period("M").dt.to_timestamp()
original_macro_preditors = original_macro_preditors.drop(columns=["yyyymm"])

# Define column names for macroeconomic predictor variables
macropredictors_names = ["d_p", "e_p", "b_m", "ntis", "tbl", "tms", "dfy", "svar"]

# Calculate macroeconomic predictor variables
macropredictors = pd.DataFrame(data={"DATE": original_macro_preditors["DATE"]})
macropredictors["d_p"] = np.log(original_macro_preditors["D12"]) - np.log(original_macro_preditors["Index"])
macropredictors["e_p"] = np.log(original_macro_preditors["E12"]) - np.log(original_macro_preditors["Index"])
macropredictors["b_m"] = original_macro_preditors["b/m"]
macropredictors["ntis"] = original_macro_preditors["ntis"]
macropredictors["tbl"] = original_macro_preditors["tbl"]
macropredictors["tms"] = original_macro_preditors["lty"] - original_macro_preditors["tbl"]
macropredictors["dfy"] = original_macro_preditors["BAA"] - original_macro_preditors["AAA"]
macropredictors["svar"] = original_macro_preditors["svar"]

# Apply missing value replacement to relevant columns in macroeconomic predictor data
macropredictors[macropredictors_names] = macropredictors.groupby(["DATE"], as_index=False, group_keys=False)[macropredictors_names].apply(replace_nas)

In [ ]:
macropredictors.head(5)

DATE       d_p       e_p       b_m      ntis     tbl     tms  \
1034 1957-03-01 -3.238565 -2.562911  0.599819  0.026600  0.0308  0.0023   
1035 1957-04-01 -3.274852 -2.597238  0.576098  0.027421  0.0307  0.0038   
1036 1957-05-01 -3.311134 -2.631567  0.564039  0.028849  0.0306  0.0042   
1037 1957-06-01 -3.309868 -2.628349  0.565877  0.030528  0.0329  0.0032   
1038 1957-07-01 -3.315439 -2.634821  0.560057  0.032346  0.0316  0.0049   

         dfy      svar  
1034  0.0077  0.000330  
1035  0.0077  0.000302  
1036  0.0078  0.000482  
1037  0.0072  0.000579  
1038  0.0074  0.000554

In [ ]:
original_data.shape

(3760033, 101)

In [ ]:
original_data.head(5)

permno       DATE     mvel1       RET     prc  SHROUT      beta  \
308642   10006 1957-03-01  0.315043  0.018405  62.250    1412  0.200000   
308643   10014 1957-03-01 -0.943236  0.000000   2.625    1487 -0.902913   
308644   10022 1957-03-01 -0.719962 -0.003846   6.375    1514 -0.118447   
308645   10030 1957-03-01  0.057711  0.077907  45.750    1207 -0.203883   
308646   10057 1957-03-01 -0.122044 -0.017730  69.250     500  0.347573   

          betasq     chmom    dolvol  ...  baspread       ill    maxret  \
308642  0.200000  0.800774  0.724596  ...  0.064272 -0.555345  0.194707   
308643 -0.904854  0.367505 -0.707502  ...  0.926276  0.920530  0.917769   
308644 -0.118447  0.601547 -0.642925  ... -0.387524  0.854305 -0.357278   
308645 -0.203883  0.137331  0.378917  ... -0.483932 -0.685904 -0.502836   
308646  0.347573 -0.181818 -0.190883  ... -0.720227 -0.381268 -0.967864   

          retvol  std_dolvol  std_turn  zerotrade  sic2   bm  bm_ia  
308642 -0.077505    0.785782  0.623819  -0.735099  37.0  0.0    0.0  
308643  0.930057    0.833175  0.540643   0.286660   NaN  0.0    0.0  
308644  0.370510    0.886256  0.366730  -0.358562   NaN  0.0    0.0  
308645 -0.686200    0.270142  0.538752  -0.801325   NaN  0.0    0.0  
308646 -0.897921    0.730806  0.599244   0.772942   NaN  0.0    0.0  

[5 rows x 101 columns]

In [ ]:
exception = original_data[exception_names]
exception.shape

(3760033, 7)

In [ ]:
data = pd.merge(original_data,macropredictors,left_on="DATE", right_on ="DATE")
data.shape

(3760033, 109)

In [ ]:
data.head(5)

permno       DATE     mvel1       RET     prc  SHROUT      beta    betasq  \
0   10006 1957-03-01  0.315043  0.018405  62.250    1412  0.200000  0.200000   
1   10014 1957-03-01 -0.943236  0.000000   2.625    1487 -0.902913 -0.904854   
2   10022 1957-03-01 -0.719962 -0.003846   6.375    1514 -0.118447 -0.118447   
3   10030 1957-03-01  0.057711  0.077907  45.750    1207 -0.203883 -0.203883   
4   10057 1957-03-01 -0.122044 -0.017730  69.250     500  0.347573  0.347573   

      chmom    dolvol  ...   bm  bm_ia       d_p       e_p       b_m    ntis  \
0  0.800774  0.724596  ...  0.0    0.0 -3.238565 -2.562911  0.599819  0.0266   
1  0.367505 -0.707502  ...  0.0    0.0 -3.238565 -2.562911  0.599819  0.0266   
2  0.601547 -0.642925  ...  0.0    0.0 -3.238565 -2.562911  0.599819  0.0266   
3  0.137331  0.378917  ...  0.0    0.0 -3.238565 -2.562911  0.599819  0.0266   
4 -0.181818 -0.190883  ...  0.0    0.0 -3.238565 -2.562911  0.599819  0.0266   

      tbl     tms     dfy     svar  
0  0.0308  0.0023  0.0077  0.00033  
1  0.0308  0.0023  0.0077  0.00033  
2  0.0308  0.0023  0.0077  0.00033  
3  0.0308  0.0023  0.0077  0.00033  
4  0.0308  0.0023  0.0077  0.00033  

[5 rows x 109 columns]

In [ ]:
data_csv = data.to_csv("cleaned_data.csv", index = False)
!cp cleaned_data.csv "drive/My Drive/MATH5470"

In [ ]:
data = pd.read_csv("drive/MyDrive/MATH5470/cleaned_data.csv")
data["sic2"] = data["sic2"].fillna(value=0)
data['DATE'] = pd.to_datetime(data['DATE'], format="%Y-%m-%d")
data["DATE"] = data["DATE"].dt.to_period("M").dt.to_timestamp()
data.head(5)

permno       DATE     mvel1       RET     prc  SHROUT      beta    betasq  \
0   10006 1957-03-01  0.315043  0.018405  62.250    1412  0.200000  0.200000   
1   10014 1957-03-01 -0.943236  0.000000   2.625    1487 -0.902913 -0.904854   
2   10022 1957-03-01 -0.719962 -0.003846   6.375    1514 -0.118447 -0.118447   
3   10030 1957-03-01  0.057711  0.077907  45.750    1207 -0.203883 -0.203883   
4   10057 1957-03-01 -0.122044 -0.017730  69.250     500  0.347573  0.347573   

      chmom    dolvol  ...   bm  bm_ia       d_p       e_p       b_m    ntis  \
0  0.800774  0.724596  ...  0.0    0.0 -3.238565 -2.562911  0.599819  0.0266   
1  0.367505 -0.707502  ...  0.0    0.0 -3.238565 -2.562911  0.599819  0.0266   
2  0.601547 -0.642925  ...  0.0    0.0 -3.238565 -2.562911  0.599819  0.0266   
3  0.137331  0.378917  ...  0.0    0.0 -3.238565 -2.562911  0.599819  0.0266   
4 -0.181818 -0.190883  ...  0.0    0.0 -3.238565 -2.562911  0.599819  0.0266   

      tbl     tms     dfy     svar  
0  0.0308  0.0023  0.0077  0.00033  
1  0.0308  0.0023  0.0077  0.00033  
2  0.0308  0.0023  0.0077  0.00033  
3  0.0308  0.0023  0.0077  0.00033  
4  0.0308  0.0023  0.0077  0.00033  

[5 rows x 109 columns]

In [ ]:
exception_names = ["permno", "DATE", "RET", "SHROUT", "sic2", "mve0", "prc"]
characteristics_names = [col for col in data.columns if col not in exception_names]
len(characteristics_names)

102

In [ ]:
sic2_onehot = pd.get_dummies(data["sic2"], prefix='sic2')
sic2_names = sic2_onehot.columns
data = pd.concat([data, sic2_onehot], axis=1)
data.shape

(3760033, 184)

In [ ]:
macropredictors_names = ["d_p", "e_p", "b_m", "ntis", "tbl", "tms", "dfy", "svar"]

In [ ]:
predictor_names = [col for col in data.columns if col not in exception_names]
len(predictor_names)

177

In [ ]:
def compute_Roos(y_predict, y_test):
    y_predict_array = np.array(y_predict).reshape(-1)
    y_test_array = np.array(y_test).reshape(-1)
    SSE = np.sum((y_predict_array - y_test_array)**2)
    SST = np.sum(y_test_array**2)
    Roos = 1 - (SSE/SST)
    return Roos

def compute_vi(model, x_train, y_train, characteristics_names, file_name):
    y_fitted = model.predict(x_train)
    Roos_base = compute_Roos(y_fitted, y_train)
    vis = np.array([])

    for item in characteristics_names:
        x_temp = x_train.copy(deep=True)
        x_temp[item] = 0
        y_fitted = model.predict(x_temp)
        Roos_item = compute_Roos(y_fitted, y_train)
        vis = np.append(vis, Roos_base-Roos_item)
    print(vis)
    with open(file_name,'a') as fd:
        writer = csv.writer(fd)
        writer.writerow(vis)

In [ ]:
from dateutil.relativedelta import relativedelta
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
import csv

train_start = datetime.strptime("1957-03-01", "%Y-%m-%d")
train_end = datetime.strptime("1974-12-31", "%Y-%m-%d")
val_start = datetime.strptime("1975-01-01", "%Y-%m-%d")
val_end = datetime.strptime("1986-12-31", "%Y-%m-%d")
test_start = datetime.strptime("1987-01-01", "%Y-%m-%d")
test_end = datetime.strptime("1987-12-31", "%Y-%m-%d")

result = pd.DataFrame(columns=["permno", "DATE", "SHROUT", "prc", "RET", "RET_pcr"])
#Roos_pcr_monthly = pd.DataFrame(columns=["DATE", "Roos_all", "Roos_top_1000","Roos_bottom_1000"])
#Roos_pcr_yearly = pd.DataFrame(columns=["DATE", "Roos_all", "Roos_top_1000","Roos_bottom_1000"])
#Roos_pls_monthly = pd.DataFrame(columns=["DATE", "Roos_all", "Roos_top_1000","Roos_bottom_1000"])
#Roos_pls_yearly = pd.DataFrame(columns=["DATE", "Roos_all", "Roos_top_1000","Roos_bottom_1000"])

# Tuning parameters
ns_pcr = [30,50,70]
#ns_pls = [1, 3, 5]

for i in range(30):
    print("test period: ",i)
    train_index = (data['DATE'] >= train_start) & (data['DATE'] <= train_end)
    val_index = (data['DATE'] >= val_start) & (data['DATE'] <= val_end)
    test_index = (data['DATE'] >= test_start) & (data['DATE'] <= test_end)


    x_train = data[train_index][predictor_names]
    y_train = data[train_index]['RET']
    x_val = data[val_index][predictor_names]
    y_val = data[val_index]['RET']
    x_test = data[test_index][predictor_names]
    y_test = data[test_index]['RET']

    models_val_pcr = []
    #models_val_pls = []
    Roos_val_pcr = []
    #Roos_val_pls = []
    for tune_index in np.arange(len(ns_pcr)):
        n_pcr = ns_pcr[tune_index]
        #n_pls = ns_pls[tune_index]
        # train pcr
        pcr = make_pipeline(StandardScaler(), PCA(n_components=n_pcr), LinearRegression())
        pcr.fit(x_train, y_train)
        y_predict_val_pcr = pcr.predict(x_val)
        Roos_val_pcr.append(compute_Roos(y_predict_val_pcr, y_val))
        models_val_pcr.append(pcr)
        # train pls
        # pls = PLSRegression(n_components=n_pls)
        # pls.fit(x_train, y_train)
        # y_predict_val_pls = pls.predict(x_val)
        # Roos_val_pls.append(compute_Roos(y_predict_val_pls, y_val))
        # models_val_pls.append(pls)
    #validation
    index_best_pcr = np.argmax(np.array(Roos_val_pcr))
    #index_best_pls = np.argmax(np.array(Roos_val_pls))
    pcr = models_val_pcr[index_best_pcr]
    #pls = models_val_pls[index_best_pls]
    n_pcr = ns_pcr[index_best_pcr]
    #n_pls = ns_pls[index_best_pls]
    # test
    y_predict_pcr = pcr.predict(x_test)
    #y_predict_pls = pls.predict(x_test)

    summary_df = data[test_index][["permno", "DATE", "SHROUT", "prc", "RET"]]
    summary_df["RET_pcr"] = y_predict_pcr
    #summary_df["RET_pls"] = y_predict_pls
    result = pd.concat([result,summary_df], axis=0)

    # variable importance computing
    if i==29: compute_vi(pcr, x_train, y_train, characteristics_names, "VI_PCA.csv")
    #compute_vi(pls, x_train, y_train, characteristics_names, "PLS_vi.csv")

    train_end = train_end + relativedelta(years=1)
    val_start = val_start + relativedelta(years=1)
    val_end = val_end + relativedelta(years=1)
    test_start = test_start + relativedelta(years=1)
    test_end = test_end + relativedelta(years=1)

result.to_csv("results_PCA.csv", index = False)
!cp results_PCA.csv "drive/My Drive/MATH5470/project"

!cp VI_PCA.csv "drive/My Drive/MATH5470/project"
#!cp PLS_vi.csv "drive/My Drive/MATH5470"

test period:  0
test period:  1
test period:  2
test period:  3
test period:  4
test period:  5
test period:  6
test period:  7
test period:  8
test period:  9
test period:  10
test period:  11
test period:  12
test period:  13
test period:  14
test period:  15
test period:  16
test period:  17
test period:  18
test period:  19
test period:  20
test period:  21
test period:  22
test period:  23
test period:  24
test period:  25
test period:  26
test period:  27
test period:  28
test period:  29


<ipython-input-8-7984322dcc7e>:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  summary_df["RET_pcr"] = y_predict_pcr


[ 4.52843134e-05 -6.84866978e-08  1.18621860e-08  1.97335198e-04
  1.17489681e-05  6.98562939e-06  2.35819600e-04  6.61253685e-04
  2.85122750e-05  1.26254930e-04  4.20584662e-05 -3.84962649e-05
 -1.70829633e-05 -4.30923411e-08  1.65731541e-06 -5.51943608e-05
  3.25296777e-05  4.53962741e-06  1.95509076e-05  2.35905145e-06
  2.89747287e-07  1.45230215e-07  1.00500227e-05 -1.21802844e-05
 -2.49817539e-06 -4.62054697e-05 -1.23103790e-05  2.90381629e-07
  9.96874474e-06 -4.61705722e-05  1.86203012e-06 -1.52132634e-05
  7.92920222e-06  1.34980058e-05  7.16916164e-07  6.29273473e-06
  3.11762710e-06 -1.65245863e-06 -6.73495895e-06  5.59521127e-06
  3.58186264e-06  9.35491074e-06 -2.46344512e-05  6.55029379e-06
 -2.88816386e-06 -1.09076539e-05  3.30567466e-06 -1.82351626e-05
  2.77927771e-06  4.41474220e-06  1.12661218e-06 -3.32982148e-06
 -5.71457217e-06  1.33662708e-06 -1.32151170e-06 -2.31434306e-08
  5.81959763e-07  3.77425632e-05  4.41922713e-05  4.04108221e-06
  3.00726194e-08  6.65383